# 🚀 IRIStool Quickstart
This short demo shows how to:
- Connect to InterSystems IRIS using IRIStool Python module
- Create and query a simple SQL table
- Create related tables with foreign keys
- Convert a Pandas DataFrame into an IRIS table
- Run SQL queries directly from Python

In [1]:
import pandas as pd
from utils.iristool import IRIStool
import dotenv
import os
from datetime import datetime

### Load environment variables (host, port, namespace, credentials)

In [2]:
dotenv.load_dotenv()
host = os.getenv("IRIS_HOST")
port = os.getenv("IRIS_PORT")
namespace = os.getenv("IRIS_NAMESPACE")
user = os.getenv("IRIS_USER")
password = os.getenv("IRIS_PASSWORD")

## 1️⃣ Connect to IRIS - Create and query a simple table

Print connection information

In [3]:
with IRIStool(host=host, port=port, namespace=namespace, username=user, password=password) as iris:
    # print connection information
    print(iris)

IRIS connection [_SYSTEM@127.0.0.1:9093/USER]


Clear existing tables

In [4]:
with IRIStool(host=host, port=port, namespace=namespace, username=user, password=password) as iris:
    # Check if a table exists
    if iris.table_exists(table_name="Employee",table_schema="Company"):
        print("Employee table found! Dropping table...")
        iris.drop_table(table_name="Employee",table_schema="Company", if_exists=True, drop_related_views=True)
    else: 
        print("Employee table not found!")
    
    # Check if a table exists
    if iris.table_exists(table_name="Employer",table_schema="Company"): 
        print("Employer table found! Dropping table...")
        iris.drop_table(table_name="Employer",table_schema="Company", if_exists=True, drop_related_views=True) 
    else: 
        print("Employer table not found!")

Employee table found! Dropping table...
table Company.Employee dropped successfully.
Employer table found! Dropping table...
table Company.Employer dropped successfully.


Create Employer table

In [5]:
with IRIStool(host=host, port=port, namespace=namespace, username=user, password=password) as iris:
    # Create the table
    iris.create_table(table_name="Employer", table_schema="Company", 
        columns={"employer_id": "INT", "name": "VARCHAR(200)", "industry": "VARCHAR(100)", "founded": "INT" }, 
        constraints=["PRIMARY KEY (employer_id)"], 
        check_exists=True 
    ) 
    
    # insert an index on industry
    iris.create_index(table_name="Employer", table_schema="Company", index_name="industry_idx", column_name="industry") 
    
    # insert rows
    iris.insert_many("Employer", [ 
        {"employer_id": 1, "name": "OpenAI", "industry": "AI Research", "founded": 2015}, 
        {"employer_id": 2, "name": "InterSystems", "industry": "Healthcare IT", "founded": 1978}, 
        {"employer_id": 3, "name": "Spotify", "industry": "Music Tech", "founded": 2006} ], 
    table_schema="Company") 
    
    # display a DataFrame with fetched data
    display(iris.fetch("SELECT * FROM Company.Employer"))

Table Company.Employer created successfully.
Index industry_idx created successfully on Company.Employer(industry).
3 row(s) added into Company.Employer.


,employer_id,name,industry,founded
0,1,OpenAI,AI Research,2015
1,2,InterSystems,Healthcare IT,1978
2,3,Spotify,Music Tech,2006


## 2️⃣ Create Child Table — Employee (with Foreign Key to Employer)

Create a DataFrame

In [6]:
# define a DataFrame
df = pd.DataFrame({
    "employee_id": [1, 2, 3],
    "name": ["Alice", "Bob", "Charlie"],
    "is_active": [True, False, True], # Boolean
    "salary": [72000.50, 55000.00, 62000.75], # Float,
    "age": [29, 34, 28], # Integer
    "hire_date": [datetime(2020,5,1), datetime(2019,8,15), datetime(2021,3,3)], # Datetime
    "department": ["IT", "HR", "Finance"], # String
    "projects_completed": [15, 8, 10], # Integer
    "employer_id": [1, 2, 3]
})

print("Original DataFrame:")
print(df.dtypes)
display(df)

Original DataFrame:
employee_id                    int64
name                          object
is_active                       bool
salary                       float64
age                            int64
hire_date             datetime64[ns]
department                    object
projects_completed             int64
employer_id                    int64
dtype: object


,employee_id,name,is_active,salary,age,hire_date,department,projects_completed,employer_id
0,1,Alice,True,72000.50,29,2020-05-01,IT,15,1
1,2,Bob,False,55000.00,34,2019-08-15,HR,8,2
2,3,Charlie,True,62000.75,28,2021-03-03,Finance,10,3


Import the dataframe automatically inferring types

In [7]:
with IRIStool(host=host, port=port, namespace=namespace, username=user, password=password) as iris:        
    fk = ["FOREIGN KEY(employer_id) REFERENCES Company.Employer(employer_id)"] 
        
    # Convert DataFrame into an IRIS table 
    iris.df_to_table(
        df=df, 
        table_name="Employee", 
        table_schema="Company", 
        primary_key="employee_id", 
        exist_ok=True, 
        drop_if_exists=True,
        constraints=fk
    )    
    
    # show column types -> Automatically infer IRIS data types from a DataFrame during conversion
    info = iris.describe_table(table_name="Employee", table_schema="Company")
    
    df_cols = pd.DataFrame(info["columns"])
    display("Columns:", df_cols)

    df_idx = pd.DataFrame(info["indexes"])
    display("Indexes:", df_idx)

Table Company.Employee created successfully.
Inserted 3 rows into Employee


'Columns:'

,TABLE_SCHEMA,TABLE_NAME,COLUMN_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,IS_NULLABLE,AUTO_INCREMENT,UNIQUE_COLUMN,PRIMARY_KEY,odbctype
0,Company,Employee,employee_id,integer,NaN,NO,NO,YES,YES,4
1,Company,Employee,name,varchar,255.0,YES,NO,NO,NO,12
2,Company,Employee,is_active,bit,NaN,YES,NO,NO,NO,-7
3,Company,Employee,salary,double,NaN,YES,NO,NO,NO,8
4,Company,Employee,age,integer,NaN,YES,NO,NO,NO,4
5,Company,Employee,hire_date,date,NaN,YES,NO,NO,NO,9
6,Company,Employee,department,varchar,255.0,YES,NO,NO,NO,12
7,Company,Employee,projects_completed,integer,NaN,YES,NO,NO,NO,4
8,Company,Employee,employer_id,integer,NaN,YES,NO,NO,NO,4


'Indexes:'

,INDEX_NAME,COLUMN_NAME,PRIMARY_KEY,NON_UNIQUE
0,EMPLOYEEPKey1,employee_id,1,0


Show foreign keys

In [8]:
with IRIStool(host=host, port=port, namespace=namespace, username=user, password=password) as iris:
    foreign_keys = iris.get_reference_from_this_table(table_name="Employee", table_schema="Company")
    display("Foreign keys:", foreign_keys)
    
    # Get reference to this table
    references = iris.get_reference_to_this_table(table_name="Employer", table_schema="Company")
    display("References:", references)

'Foreign keys:'

,constraint_name,referenced_table,referenced_schema,table_schema,table_name,column_name,referenced_column_name
0,EMPLOYEEFKey2,Employer,Company,Company,Employee,employer_id,employer_id


'References:'

,constraint_name,referenced_table,referenced_schema,table_schema,table_name,column_name,referenced_column_name
0,EMPLOYEEFKey2,Employer,Company,Company,Employee,employer_id,employer_id


Fetch data from IRIS database

In [9]:
with IRIStool(host=host, port=port, namespace=namespace, username=user, password=password) as iris:        
    desired_age = 30
    print(f"Retrieving data of employees with age greater than {desired_age}...")
    employees = iris.fetch("SELECT * FROM Company.Employee WHERE Age > ?", [desired_age])
    display(employees)    
    print("Describing dataframe:")
    display(employees.describe())

Retrieving data of employees with age greater than 30...


,employee_id,name,is_active,salary,age,hire_date,department,projects_completed,employer_id
0,2,Bob,0,55000.0,34,2019-08-15,HR,8,2


Describing dataframe:


,employee_id,is_active,salary,age,projects_completed,employer_id
count,1.0,1.0,1.0,1.0,1.0,1.0
mean,2.0,0.0,55000.0,34.0,8.0,2.0
std,NaN,NaN,NaN,NaN,NaN,NaN
min,2.0,0.0,55000.0,34.0,8.0,2.0
25%,2.0,0.0,55000.0,34.0,8.0,2.0
50%,2.0,0.0,55000.0,34.0,8.0,2.0
75%,2.0,0.0,55000.0,34.0,8.0,2.0
max,2.0,0.0,55000.0,34.0,8.0,2.0


## Thank you!